In [1]:
import time

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau

from HW_base import evaluate_build, focus_build
from HW_keras import hw_layer

import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [4]:
x_min = np.min(x_train, axis=(1, 2), keepdims=True)
x_max = np.max(x_train, axis=(1, 2), keepdims=True)
x_train = (x_train - x_min) / (x_max - x_min)

In [5]:
x_min = np.min(x_test, axis=(1, 2), keepdims=True)
x_max = np.max(x_test, axis=(1, 2), keepdims=True)
x_test = (x_test - x_min) / (x_max - x_min)

In [6]:
evaluate_list = [evaluate_build(x_test[..., i], 8) for i in range(x_test.shape[-1])]
evaluate_focus_list = [focus_build(evaluate, 0.8) for evaluate in evaluate_list]

evaluate_num:   8,focus:0.8000: 8it [00:00, ?it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, 8012.04it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, 8027.38it/s]


In [7]:
model = Sequential()

model.add(hw_layer(evaluate_focus_list, input_shape=(32, 32, 3), name='hw_layer'))
model.add(Conv2D(10, (32, 32)))

# model.add(Conv2D(10, (32, 32), input_shape=(32, 32, 3)))
model.add(Flatten())
model.add(Activation('softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hw_layer (hw_layer)          (None, 32, 32, 24)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 1, 1, 10)          245770    
_________________________________________________________________
flatten (Flatten)            (None, 10)                0         
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 245,770
Trainable params: 245,770
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_name = time.strftime('CIFAR10_KERAS_HWNET%Y%m%d%H%M%S')

callback_list = [
#     ReduceLROnPlateau(monitor='loss', factor=0.7,  patience=8, min_lr=1e-7, verbose=True),
    EarlyStopping(monitor='loss', patience=32),
    ModelCheckpoint('models/%s_best.h5'%model_name, monitor='loss', save_best_only=True, verbose=False),
    TensorBoard(log_dir='./Log/%s'%model_name)
]

model.fit(x_train, y_train, epochs=200, batch_size=500, validation_data=(x_test, y_test), callbacks=callback_list)

Epoch 1/200
100/100 [==============================] - 4s 15ms/step - loss: 1.8589 - accuracy: 0.3605 - val_loss: 1.7421 - val_accuracy: 0.3996
Epoch 2/200
100/100 [==============================] - 1s 10ms/step - loss: 1.5997 - accuracy: 0.4541 - val_loss: 1.7079 - val_accuracy: 0.4217
Epoch 3/200
100/100 [==============================] - 1s 10ms/step - loss: 1.4817 - accuracy: 0.5000 - val_loss: 1.7608 - val_accuracy: 0.3964
Epoch 4/200
100/100 [==============================] - 1s 10ms/step - loss: 1.3892 - accuracy: 0.5338 - val_loss: 1.7577 - val_accuracy: 0.3934
Epoch 5/200
100/100 [==============================] - 1s 10ms/step - loss: 1.3168 - accuracy: 0.5649 - val_loss: 1.7784 - val_accuracy: 0.3891
Epoch 6/200
100/100 [==============================] - 1s 10ms/step - loss: 1.2541 - accuracy: 0.5876 - val_loss: 1.8236 - val_accuracy: 0.3994
Epoch 7/200
100/100 [==============================] - 1s 10ms/step - loss: 1.2065 - accuracy: 0.6066 - val_loss: 1.8313 - val_accuracy:

100/100 [==============================] - 1s 10ms/step - loss: 0.3544 - accuracy: 0.9373 - val_loss: 2.9918 - val_accuracy: 0.3196
Epoch 58/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3456 - accuracy: 0.9433 - val_loss: 3.0287 - val_accuracy: 0.3164
Epoch 59/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3468 - accuracy: 0.9394 - val_loss: 3.0735 - val_accuracy: 0.3170
Epoch 60/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3357 - accuracy: 0.9448 - val_loss: 3.0676 - val_accuracy: 0.3166
Epoch 61/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3269 - accuracy: 0.9488 - val_loss: 3.0990 - val_accuracy: 0.3134
Epoch 62/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3212 - accuracy: 0.9515 - val_loss: 3.1150 - val_accuracy: 0.3156
Epoch 63/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3132 - accuracy: 0.9532 - val_loss: 3.1322 - val_accuracy: 0.312